In [88]:
pip install json5

Note: you may need to restart the kernel to use updated packages.


In [89]:
import pandas as pd 
import numpy as np
import requests
import boto3
import os
import json
from io import BytesIO
import base64
from bs4 import BeautifulSoup
import json5

charts_path = "/Users/finn/Documents/GitHub/GrowthDiagnostics/charts"

In [3]:
def upload_series(df, name, bucket="eco-temp-cache"):
    csv = df.to_csv(index=False, date_format='%Y-%m-%d')
    s3 = boto3.client('s3')
    res = s3.put_object(Bucket=bucket, Key=f"gd/{name}.csv", Body=csv)
    return f"https://eco-temp-cache.s3.eu-west-2.amazonaws.com/gd/{name}.csv"


In [115]:
def prepare_spec(data_url, path, title, filter=None, parent_path="charts", multi_series=False, yAxisExpr=None):
    spec = {
        "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
        "title": {"text": title},
        "transform": [{"filter": "datum.value>0"}],
        "data": {
            "url": data_url,
        },
        "mark": {"type": "line"},
        "encoding": {
            "x": {
                "field": "date",
                "type": "temporal",
                "axis": {"title": "", "titleColor": "#122B39"}
            },
            "y": {
                "field": "value",
                "type": "quantitative",
                "axis": {"title": "", "titleColor": "#122B39"}
            }
        }
    }
    if filter:
        spec['transform'].append({"filter": filter})
    if multi_series:
        spec['encoding']['color'] = {"field": "series", "type": "nominal"}
    if yAxisExpr:
        spec['encoding']['y']['axis']['labelExpr'] = yAxisExpr
    with open(f"{parent_path}/{path}.json", "w") as f:
        f.write(json.dumps(spec))

In [111]:
def get_stats(df):
    df = df.sort_values('date')
    # get the most recent value and it's date
    latest = df.iloc[-1]
    latest_date = latest.date
    latest_value = latest.value
    # get the as close to 1 year ago as possible
    year_ago = df[df.date<=df.date.max()-pd.DateOffset(years=1)].iloc[-1]
    year_ago_date = year_ago.date
    year_ago_value = year_ago.value
    # get the average for the last 10 years (or as many as we have)
    avg = df[df.date>df.date.max()-pd.DateOffset(years=10)].value.mean()
    avg_year_count = df.iloc[-1].date.year-df[df.date>df.date.max()-pd.DateOffset(years=10)].iloc[0].date.year
    return {
        'latest': {'date': latest_date, 'value': latest_value},
        'latest_date': latest_date,
        'year_ago': {'date': year_ago_date, 'value': year_ago_value},
        'year_ago_date': year_ago_date,
        'avg': avg,
        'avg_year_count': avg_year_count
    }

def human_change(val1, val2, date1, date2, date_format="%b %Y", val_format=".1%", change_format=".3f", change_multiplier=1, change_units=""):
    print(f"val1: {val1}, val2: {val2}, date1: {date1}, date2: {date2}")
    if np.round(val1, 3) == np.round(val2, 3):
        return f"unchanged from {date2.strftime(date_format)}"
    if val1 > val2:
        return f"up {(val1-val2)*change_multiplier:{change_format}}{change_units} from {date2.strftime(date_format)} when it was {val2:{val_format}}"
    if val1 < val2:
        return f"down {(val1-val2)*change_multiplier:{change_format}}{change_units} from {date2.strftime(date_format)} when it was {val2:{val_format}}"  

def temporal_description(data, name="it", preamble="", val_format=".1%", change_format=".3f", change_multiplier=1, change_units="", date_format="%b %Y"):
    if len(preamble)>0 and preamble[-1]!=" ":
        preamble += " "
    stats = get_stats(data)
    return f"{preamble}In {stats['latest']['date'].strftime('%B %Y')}, {name} was {stats['latest']['value']:.1%}. This is {human_change(stats['latest']['value'], stats['year_ago']['value'], stats['latest']['date'], stats['year_ago']['date'], date_format, val_format, change_format, change_multiplier, change_units)}. The average for the last {stats['avg_year_count']} years is {stats['avg']:.1%}."

def upload_description(description, name, bucket="eco-temp-cache"):
    s3 = boto3.client('s3')
    res = s3.put_object(Bucket=bucket, Key=f"gd/descriptions/{name}.txt", Body=json.dumps(description))
    return f"https://eco-temp-cache.s3.eu-west-2.amazonaws.com/gd/descriptions/{name}.txt"


# Gilt Yields

In [116]:
def fetch_and_parse_dmo(report_code, start_year, start_month, end_year, end_month):
    if end_year == "max":
        end_year = pd.Timestamp.now().year
    url = f"https://www.dmo.gov.uk/umbraco/surface/DataExport/GetDataExport?reportCode={report_code}&exportFormatValue=xls&parameters=%26Start%20Month%3D{start_month}%26Start%20Year%3D{start_year}%26End%20Month%3D{end_month}%26End%20Year%3D{end_year}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:123.0) Gecko/20100101 Firefox/123.0",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
        "Accept-Language": "en-GB,en;q=0.5",
        "Upgrade-Insecure-Requests": "1",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "same-origin",
        "Sec-Fetch-User": "?1",
        "Referer": "https://www.dmo.gov.uk/data/ExportReport?reportCode=D4H"
    }
    
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        # Read the .xls file content into a pandas DataFrame
        df = pd.read_excel(BytesIO(response.content), skiprows=3, engine="xlrd")
        return df
    else:
        return f"Error fetching data: Status code {response.status_code}"



In [117]:
gilt_yield_ids = {
    "report_code": "D4H",
}

raw_gilt_yield_df = fetch_and_parse_dmo(gilt_yield_ids["report_code"], 1998, 1, "max", 12)

# 1998-04-01 00:00:00
gilt_yield_df = raw_gilt_yield_df.copy()
gilt_yield_df["date"] = pd.to_datetime(gilt_yield_df["Month"], format="%Y-%m-%d %H:%M:%S", errors="coerce")
gilt_yield_df = gilt_yield_df.dropna(subset=["date"]).drop(columns=["Month"])
gilt_yield_df = gilt_yield_df.melt(id_vars=["date"], var_name="series", value_name="value")
gilt_yield_df.series = gilt_yield_df.series.str.strip()

gilt_yield_df.series = gilt_yield_df.series.map({
    "Short": "5 Year",
    "Medium": "10 Year",
    "Long": "30 Year",
    "Ultra Long": "50 Year"
})

gilt_yield_df = gilt_yield_df.dropna()
gilt_yield_df['value'] = pd.to_numeric(gilt_yield_df['value'], errors='coerce')/100

gilt_yield_url = upload_series(gilt_yield_df, "gilt_yield")

prepare_spec(gilt_yield_url, "finance_gilt_yield", "UK Gilt Yields", multi_series=True, parent_path=charts_path)

In [118]:
stats = get_stats(gilt_yield_df)
description = temporal_description(gilt_yield_df.query("series=='10 Year'"), 
                                   "the 10 year UK gilt yield", 
                                   "UK Gilt Yields are the interest rates paid by the UK government on its debt.", 
                                   val_format=".2%",
                                   change_format=".3f",
                                   change_multiplier=100,
                                   change_units="pp",
                                   date_format="%b %Y")
description_url = upload_description(description, "finance_gilt_yield")
print(description)

val1: 0.04040603523809, val2: 0.03546645, date1: 2024-02-01 00:00:00, date2: 2023-02-01 00:00:00
UK Gilt Yields are the interest rates paid by the UK government on its debt. In February 2024, the 10 year UK gilt yield was 4.0%. This is up 0.494pp from Feb 2023 when it was 3.55%. The average for the last 10 years is 1.7%.


# BoE Bank Rate

In [129]:
url = "https://www.bankofengland.co.uk/boeapps/database/Bank-Rate.asp"
req = requests.get(url)

soup = BeautifulSoup(req.text, 'html.parser')
# find the table with stats-table class
table = soup.find('table', id='stats-table')
parsed_data = []
for row in table.find_all('tr'):
    tds = row.find_all('td')
    if len(tds) == 0:
        continue
    parsed_data.append({
        "date": pd.to_datetime(tds[0].text, format="%d %b %y", errors="coerce"),
        "value": tds[1].text.strip()
    })
boe_df = pd.DataFrame(parsed_data)
boe_df['value'] = pd.to_numeric(boe_df['value'], errors='coerce')/100
boe_df = boe_df.dropna()

boe_url = upload_series(boe_df, "boe")

prepare_spec(boe_url, "finance_boe", "BoE Base Rate", parent_path=charts_path, yAxisExpr="format(datum.value, '.0%')")


In [110]:
get_stats(boe_df)

{'latest': {'date': Timestamp('1975-01-20 00:00:00'), 'value': 0.1125},
 'latest_date': Timestamp('1975-01-20 00:00:00'),
 'year_ago': {'date': Timestamp('1975-01-20 00:00:00'), 'value': 0.1125},
 'year_ago_date': Timestamp('1975-01-20 00:00:00'),
 'avg': 0.020578947368421054,
 'avg_year_count': -48}

In [112]:
stats = get_stats(boe_df)
description = temporal_description(boe_df, 
                                   "the Bank of England base rate", 
                                   "The Bank of England base rate is the interest rate at which the Bank of England lends to commercial banks.", 
                                   val_format=".2%",
                                   change_format=".3f",
                                   change_multiplier=100,
                                   change_units="pp",
                                   date_format="%b %Y")
description_url = upload_description(description, "finance_boe")
print(description)

val1: 0.0525, val2: 0.0125, date1: 2023-08-03 00:00:00, date2: 2022-06-16 00:00:00
The Bank of England base rate is the interest rate at which the Bank of England lends to commercial banks. In August 2023, the Bank of England base rate was 5.2%. This is up 4.000pp from Jun 2022 when it was 1.25%. The average for the last 7 years is 2.1%.


# BoE Mortgage Rate

In [120]:
def fetch_bank_of_england_mortage_data():
    url = "https://www.bankofengland.co.uk/boeapps/database/fromshowcolumns.asp?Travel=NIxSUx&FromSeries=1&ToSeries=50&DAT=RNG&FD=1&FM=Jan&FY=1963&TD=26&TM=Mar&TY=2024&FNY=&CSVF=TT&html.x=1375&html.y=139&C=5OK&Filter=N"
    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:124.0) Gecko/20100101 Firefox/124.0",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
        "Accept-Language": "en-GB,en;q=0.5",
        "Upgrade-Insecure-Requests": "1",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "none",
        "Sec-Fetch-User": "?1"
    }

    response = requests.get(url, headers=headers)

    # Depending on what you need from the response, you might return the content, JSON data, etc.
    # Here, the content of the response is returned directly
    return response.content


In [121]:
content = fetch_bank_of_england_mortage_data()

In [128]:
soup = BeautifulSoup(content, 'html.parser')
# find the script tag whose text starts with AmCharts.makeChart("chartdiv",
script = soup.find('script', text=lambda x: x and x.strip().startswith('AmCharts.makeChart("chartdiv",'))
script = script.text
spec = script.split('AmCharts.makeChart("chartdiv",')[1]
# delete the last ); from the string
spec = "".join(spec.rsplit(");", 1))
spec = json5.loads(spec)
data = spec.get("dataProvider")
parsed_data = [
    {
        "date": pd.to_datetime(d["Date"], format="%d-%m-%Y"),
        "value": pd.to_numeric(d["Value"], errors="coerce")/100
    }
    for d in data
]
mortgage_df = pd.DataFrame(parsed_data)

mortgage_url = upload_series(mortgage_df, "mortgage")

# prepare the spec
prepare_spec(mortgage_url, "finance_mortgage", "Average weighted mortgage rate, CFMHSDE ", parent_path=charts_path, yAxisExpr="format(datum.value, '.0%')")


/var/folders/07/g_yq2xsn0mvb4xnyvr18jjmw0000gn/T/ipykernel_71191/1429386138.py:3: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  script = soup.find('script', text=lambda x: x and x.strip().startswith('AmCharts.makeChart("chartdiv",'))


In [123]:
stats = get_stats(mortgage_df)
description = temporal_description(mortgage_df, 
                                   "the average weighted residential mortgage rate", 
                                   "", 
                                   val_format=".2%",
                                   change_format=".3f",
                                   change_multiplier=100,
                                   change_units="pp",
                                   date_format="%b %Y")
description_url = upload_description(description, "finance_mortgage")
print(description)

val1: 0.0342, val2: 0.0255, date1: 2024-01-31 00:00:00, date2: 2023-01-31 00:00:00
In January 2024, the average weighted residential mortgage rate was 3.4%. This is up 0.870pp from Jan 2023 when it was 2.55%. The average for the last 10 years is 2.6%.


# Lending Growth
https://www.bankofengland.co.uk/boeapps/database/fromshowcolumns.asp?Travel=NIxSTxTAxSUx&FromSeries=1&ToSeries=50&DAT=ALL&FNY=&CSVF=TT&html.x=163&html.y=43&C=E6B&Filter=N
Quarterly 12 month growth rate of M4 (monetary financial institutions' sterling M4 liabilities to private sector) (in percent) seasonally adjusted 
LPQVQJW

In [124]:
def fetch_bank_of_england_m4_data():
    url = "https://www.bankofengland.co.uk/boeapps/database/fromshowcolumns.asp?Travel=NIxSTxTAxSUx&FromSeries=1&ToSeries=50&DAT=ALL&FNY=&CSVF=TT&html.x=163&html.y=43&C=E6B&Filter=N"
    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:124.0) Gecko/20100101 Firefox/124.0",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
        "Accept-Language": "en-GB,en;q=0.5",
        "Upgrade-Insecure-Requests": "1",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "none",
        "Sec-Fetch-User": "?1"
    }

    response = requests.get(url, headers=headers)

    # Depending on what you need from the response, you might return the content, JSON data, etc.
    # Here, the content of the response is returned directly
    return response.content


In [127]:
content = fetch_bank_of_england_m4_data()
soup = BeautifulSoup(content, 'html.parser')
# find the script tag whose text starts with AmCharts.makeChart("chartdiv",
script = soup.find('script', text=lambda x: x and x.strip().startswith('AmCharts.makeChart("chartdiv",'))
script = script.text
spec = script.split('AmCharts.makeChart("chartdiv",')[1]
# delete the last ); from the string
spec = "".join(spec.rsplit(");", 1))
spec = json5.loads(spec)
data = spec.get("dataProvider")
parsed_data = [
    {
        "date": pd.to_datetime(d["Date"], format="%d-%m-%Y"),
        "value": pd.to_numeric(d["Value"], errors="coerce")/100
    }
    for d in data
]

m4_df = pd.DataFrame(parsed_data)
m4_url = upload_series(m4_df, "m4")

prepare_spec(m4_url, "finance_m4", "M4, Private Sector Libailities, SA 12mo Growth", parent_path=charts_path, yAxisExpr="format(datum.value, '.0%')")


/var/folders/07/g_yq2xsn0mvb4xnyvr18jjmw0000gn/T/ipykernel_71191/2480324831.py:4: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  script = soup.find('script', text=lambda x: x and x.strip().startswith('AmCharts.makeChart("chartdiv",'))


In [126]:
stats = get_stats(m4_df)
description = temporal_description(m4_df, 
                                   "private sector liability growth", 
                                   "", 
                                   val_format=".2%",
                                   change_format=".3f",
                                   change_multiplier=100,
                                   change_units="pp",
                                   date_format="%b %Y")
description_url = upload_description(description, "finance_m4")
print(description)

val1: -0.009000000000000001, val2: 0.016, date1: 2023-12-31 00:00:00, date2: 2022-12-31 00:00:00
In December 2023, private sector liability growth was -0.9%. This is down -2.500pp from Dec 2022 when it was 1.60%. The average for the last 9 years is 3.5%.


In [74]:
# CFMHSDE - Monthly average of UK resident monetary financial institutions' (excl. Central Bank) sterling weighted average interest rate, loans secured on dwellings to households (in percent) not seasonally adjusted
# https://www.bankofengland.co.uk/boeapps/database/fromshowcolumns.asp?Travel=NIxSUx&FromSeries=1&ToSeries=50&DAT=RNG&FD=1&FM=Jan&FY=1963&TD=26&TM=Mar&TY=2024&FNY=&CSVF=TT&html.x=1375&html.y=139&C=5OK&Filter=N

url = "https://www.bankofengland.co.uk/boeapps/database/fromshowcolumns.asp?Travel=NIxSUx&FromSeries=1&ToSeries=50&DAT=RNG&FD=1&FM=Jan&FY=1963&TD=26&TM=Mar&TY=2024&FNY=&CSVF=TT&html.x=1375&html.y=139&C=5OK&Filter=N"
req = requests.get(url)
with open("boe.html", "w") as f:
    f.write(req.text)
soup = BeautifulSoup(req.text, 'html.parser')
# find the table with stats-table class
table = soup.find('table', id='stats-table')
parsed_data = []
for row in table.find_all('tr'):
    tds = row.find_all('td')
    if len(tds) == 0:
        continue
    parsed_data.append({
        "date": pd.to_datetime(tds[0].text, format="%d %b %y", errors="coerce"),
        "value": tds[1].text.strip()
    })
boe_df = pd.DataFrame(parsed_data)
boe_df['value'] = pd.to_numeric(boe_df['value'], errors='coerce')/100
boe_df = boe_df.dropna()


AttributeError: 'NoneType' object has no attribute 'find_all'

In [70]:
soup

ï»¿<!DOCTYPE html>

<html class="no-js" lang="en">
<head>
<meta charset="utf-8"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-type"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<meta content="telephone=no" name="format-detection"/>
<title>Sorry | Bank of England</title>
<link href="/error/files/boe.vendor.min.css?v=1" rel="stylesheet"/>
<link href="/error/files/BoE.min.css?v=3" rel="stylesheet"/>
<script>
        var el = document.getElementsByTagName('html')[0];
        if (el.classList) { el.classList.remove('no-js') } else { el.className = el.className.replace('no-js', '') }
    </script>
<!-- Google Tag Manager -->
<script>

        var CheckCookies = {
            getCookie: function (cname) {
                var name = cname + "=";
                var decodedCookie = decodeURIComponent(document.cookie);
                var ca = decodedCookie.split(';');
                for (var i 

In [67]:
boe_df

,date,value
0,2023-08-03,0.0525
1,2023-06-22,0.0500
2,2023-05-11,0.0450
3,2023-03-23,0.0425
4,2023-02-02,0.0400
...,...,...
247,1975-03-10,0.1025
248,1975-02-17,0.1050
249,1975-02-10,0.1075
250,1975-01-27,0.1100


In [64]:
table

In [17]:

if response.status_code == 200:
    # Decode the base64-encoded content
    decoded_content = base64.b64decode(response.content)
    
    # Read the .xls file content into a pandas DataFrame
    df = pd.read_excel(BytesIO(response.content), engine='xlrd')

In [18]:
df

,Data Date: 26-Mar-2024,GILT MARKET,HISTORICAL MONTHLY GILT YIELDS,Unnamed: 3,Unnamed: 4
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,Month,Short,Medium,Long,Ultra-Long
3,1998-04-01 00:00:00,5.91,5.7,5.71,NaN
4,1998-05-01 00:00:00,5.82,5.57,5.55,NaN
...,...,...,...,...,...
314,NaN,NaN,NaN,NaN,NaN
315,NaN,NaN,NaN,NaN,NaN
316,Note:,1. The historical monthly average gilt yields...,NaN,NaN,NaN
317,NaN,NaN,NaN,NaN,NaN


In [14]:
with open("temp.xls", "wb") as f:
    f.write(response.content)